In [ ]:
# Technically this code should scrape it for all 20 dataframes

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import traceback
import os

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

# Function to scrape reviews with a limit of 1000 reviews per movie
def scrape_reviews(url):
    driver.get(url)
    reviews_data = []
    try:
        while len(reviews_data) < 1000:  # Limit to 1000 reviews
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#main > section > div.lister > div.lister-list")))
            time.sleep(5)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            review_elements = soup.select("#main > section > div.lister > div.lister-list > div")
            if not review_elements:
                break
            for review_element in review_elements:
                if len(reviews_data) >= 1000:  # Check if we have already collected 1000 reviews
                    break
                try:
                    individual_rating_element = review_element.select_one(".ipl-ratings-bar > span > span:nth-child(2)")
                    individual_rating = int(individual_rating_element.text.strip()) if individual_rating_element else None
                    review_data = {
                        'Review Date': review_element.select_one(".review-date").text.strip(),
                        'Review Title': review_element.select_one(".title").text.strip(),
                        'Username': review_element.select_one(".display-name-link > a").text.strip(),
                        'Helpfulness': review_element.select_one(".actions.text-muted").text.strip(),
                        'Helpful Votes': int(re.findall(r'(\d+)', review_element.select_one(".actions.text-muted").text.strip())[0]),
                        'Total Votes': int(re.findall(r'(\d+)', review_element.select_one(".actions.text-muted").text.strip())[1]),
                        'Individual Rating': individual_rating,
                        'Review Text': review_element.select_one(".text.show-more__control").text.strip(),
                        'Spoiler Warning': review_element.select_one(".spoiler-warning").text.strip() if review_element.select_one(".spoiler-warning") else ""
                    }
                    reviews_data.append(review_data)
                except Exception as e:
                    print(f"Error occurred while extracting review data: {e}")
                    traceback.print_exc()
            load_more_button = driver.find_elements(By.ID, "load-more-trigger")
            if load_more_button and len(reviews_data) < 1000:  # Check if we can load more reviews
                driver.execute_script("arguments[0].click();", load_more_button[0])
                time.sleep(5)
            else:
                break
    except Exception as e:
        print(f"Error occurred: {e}")
        traceback.print_exc()
    finally:
        return reviews_data

# Directory to save CSV files
save_dir = 'C:/Users/oneti/OneDrive/Documents/GitHub/IMDb_movie_review_sentiment_analysis/20_movies_datasets_with_Scrapped_data'

# Main loop to iterate through dataframes
for i in range(1, 21):  # Assuming there are 20 dataframes
    df_name = f'imdb_movies_df_{i}_url'
    df = eval(df_name)  # Dynamically get the dataframe
    all_reviews = []
    for index, row in df.iterrows():
        movie_reviews = scrape_reviews(row['User Reviews URL'])
        for review in movie_reviews:
            movie_data = row.to_dict()
            movie_data['Rating'] = movie_data.get('Rating')  # Ensure the 'Rating' key is from the dataframe
            review.update(movie_data)  # Add movie data to each review
        all_reviews.extend(movie_reviews)
    
    # Convert to DataFrame
    final_df = pd.DataFrame(all_reviews)
    
    # Save to CSV
    csv_file_path = os.path.join(save_dir, f'all_scrapped_data_{i}.csv')
    final_df.to_csv(csv_file_path, index=False)
    print(f"Dataframe {i} reviews saved to {csv_file_path}")

driver.quit()